# Imports

In [1]:
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt

sys.path.append("hw2/python/")
import needle as ndl
from needle.autograd import Tensor

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%load_ext lab_black

In [4]:
def get_tensor(*shape, entropy=1):
    np.random.seed(np.prod(shape) * len(shape) * entropy)
    return Tensor(np.random.randint(0, 100, size=shape) / 20, dtype="float32")

In [5]:
def get_int_tensor(*shape, low=0, high=10, entropy=1):
    np.random.seed(np.prod(shape) * len(shape) * entropy)
    return ndl.Tensor(np.random.randint(low, high, size=shape))

# nn.Linear

In [187]:
def linear_backward(lhs_shape, rhs_shape):
    np.random.seed(199)
    f = ndl.nn.Linear(*lhs_shape)
    global w
    w = torch.tensor(
        f.weight.detach().numpy().T, requires_grad=True, dtype=torch.float32
    )
    # f.bias.data = ndl.Tensor([[0.45, 0.75, 3.2, 1.4, 4.45]])
    x = get_tensor(*rhs_shape)
    # (f(x)**2).sum().backward()
    # f(x).backward()
    # return x.grad.cached_data

In [188]:
w = 0

In [189]:
linear_backward((10, 5), (1, 5))

In [190]:
w

tensor([[ 0.2398,  0.0695,  0.6817, -0.5950,  0.1376,  0.5538, -0.4724, -0.0554,
         -0.6448, -0.2573],
        [ 0.7468,  0.1851, -0.7049, -0.6065,  0.0610,  0.5069,  0.5349,  0.1249,
          0.4423,  0.0860],
        [-0.1551, -0.2605, -0.1237,  0.6068, -0.6507, -0.0049, -0.2992,  0.7216,
         -0.4177,  0.3573],
        [ 0.5332,  0.7452,  0.5112, -0.0863, -0.2221,  0.4689, -0.3573, -0.3319,
         -0.6078, -0.2006],
        [-0.0560, -0.3702,  0.1250,  0.0606, -0.2927, -0.0301,  0.5436,  0.5734,
          0.7111,  0.5825]], requires_grad=True)

In [191]:
np.random.seed(199)
x = torch.tensor(get_tensor(1, 10).numpy(), requires_grad=True)
lin = nn.Linear(10, 5)
lin.bias.data = torch.tensor([4.95, 3.9, 3.05, 0.8, 3.65])
lin.weight.data = w

In [192]:
x

tensor([[4.9500, 4.5000, 0.7500, 4.7500, 1.4000, 4.5000, 0.4500, 1.0000, 3.7500,
         1.1000]], requires_grad=True)

In [193]:
y1 = lin(x)
y1

tensor([[3.8499, 9.5050, 2.3803, 5.5726, 5.6684]], grad_fn=<AddmmBackward0>)

In [194]:
y2 = y1**2

In [195]:
y3 = y2.sum()
y3

tensor(174.0171, grad_fn=<SumBackward0>)

In [196]:
y3.backward()

In [197]:
x.grad

tensor([[ 20.6115,   6.9209,  -1.6256, -13.4977,  -6.6728,  18.7621,   7.2866,
           8.1854,   2.7413,   5.7237]])

In [198]:
b_ndl, w_ndl, x

NameError: name 'b_ndl' is not defined

In [199]:
get_tensor(5)

needle.Tensor([4.95 3.9  3.05 0.8  3.65])

In [200]:
w_ndl = ndl.Tensor(w.T.detach().numpy(), requires_grad=True)
b_ndl = ndl.Tensor(lin.bias.data.numpy(), requires_grad=True)
x_ndl = ndl.Tensor(x.detach().numpy(), requires_grad=True)
w_ndl.shape, b_ndl.shape, x_ndl.shape

((10, 5), (5,), (1, 10))

In [201]:
y1 = x_ndl @ w_ndl + ndl.broadcast_to(ndl.reshape(b_ndl, (1, 5)), (1, 5))
y2 = y1**2
y3 = ndl.summation(y2)
y3

needle.Tensor(174.01709)

In [202]:
y3.backward()

In [203]:
x.grad

tensor([[ 20.6115,   6.9209,  -1.6256, -13.4977,  -6.6728,  18.7621,   7.2866,
           8.1854,   2.7413,   5.7237]])

# LogSumExp

In [411]:
np.array([2, 2])

1

In [405]:
x = get_tensor(2, 2, 2)
y = ndl.logsumexp(x, None)

x_torch = torch.tensor(x.detach().numpy(), requires_grad=True)
y_torch = torch.logsumexp(x_torch, (0, 1, 2))

torch.allclose(torch.tensor(y.cached_data), y_torch)

True

In [406]:
y

needle.Tensor([5.1138697])

In [407]:
y_torch

tensor(5.1139, grad_fn=<LogsumexpBackward0>)

In [44]:
rows = 5
classes = 10

x = get_tensor(rows, classes)
y = ndl.logsumexp(x)
y.backward()

x_torch = torch.tensor(x.detach().numpy(), requires_grad=True)
y_torch = torch.logsumexp(x_torch, (0, 1))
y_torch.backward()

torch.allclose(x_torch.grad.data, torch.tensor(x.grad.cached_data))

LogSumExp gradient


True

In [45]:
rows = 50
classes = 100

x = get_tensor(rows, classes)
y = ndl.logsumexp(x)
y.backward()

x_torch = torch.tensor(x.detach().numpy(), requires_grad=True)
y_torch = torch.logsumexp(x_torch, (0, 1))
y_torch.backward()

torch.allclose(x_torch.grad.data, torch.tensor(x.grad.cached_data))

LogSumExp gradient


True

In [46]:
rows = 5
classes = 10

x = get_tensor(rows, classes)
y = ndl.logsumexp(x, (1,))
y.backward()

x_torch = torch.tensor(x.detach().numpy(), requires_grad=True)
y_torch = torch.logsumexp(x_torch, (1,))
y_torch.backward(torch.ones_like(y_torch))

torch.allclose(x_torch.grad.data, torch.tensor(x.grad.cached_data))

LogSumExp gradient


True

In [47]:
rows = 5
classes = 10

x = get_tensor(rows, classes)
y = ndl.logsumexp(x, (0,))
y.backward()

x_torch = torch.tensor(x.detach().numpy(), requires_grad=True)
y_torch = torch.logsumexp(x_torch, (0,))
y_torch.backward(torch.ones_like(y_torch))

torch.allclose(x_torch.grad.data, torch.tensor(x.grad.cached_data))

LogSumExp gradient


True

# SoftmaxLoss

In [66]:
nn.Dropout()

Dropout(p=0.5, inplace=False)

In [67]:
x_torch.bernoulli??

Docstring:
bernoulli(*, generator=None) -> Tensor

Returns a result tensor where each :math:`\texttt{result[i]}` is independently
sampled from :math:`\text{Bernoulli}(\texttt{self[i]})`. :attr:`self` must have
floating point ``dtype``, and the result will have the same ``dtype``.

See :func:`torch.bernoulli`
Type:      builtin_function_or_method


In [65]:
rows = 5
classes = 10
x = get_tensor(rows, classes)
y = get_int_tensor(rows, low=0, high=classes)
f = ndl.nn.SoftmaxLoss()
loss = f(x, y)
loss.backward()

x_torch = torch.tensor(x.detach().numpy(), requires_grad=True)
y_torch = torch.tensor(y.detach().numpy().astype("float"), requires_grad=True).long()
f_torch = nn.CrossEntropyLoss()
loss_torch = f_torch(x_torch, y_torch)
loss_torch.backward()

torch.allclose(x_torch.grad.data, torch.tensor(x.grad.cached_data, dtype=torch.float32))

True

# BatchNorm

In [5]:
x = get_tensor(5, 4)
bn = ndl.nn.BatchNorm1d(4)
weight = get_tensor(4, entropy=42)
bias = get_tensor(4, entropy=1337)
bn.weight.data = weight
bn.bias.data = bias
y_ndl = bn(x)
y_ndl

needle.Tensor([[ 6.8662453   3.2144845   0.3852415   3.3185995 ]
 [-0.7392421  -0.78740615  6.588537    0.83170843]
 [ 7.4094944  -3.8132255   2.9521224   6.999198  ]
 [-1.2824903   2.4336276   5.9468174   8.690283  ]
 [ 8.495994    3.7025197   4.877283    4.9102097 ]])

In [212]:
bn.running_mean

needle.Tensor([0.32500002 0.33900002 0.211      0.26700002])

In [213]:
bn.running_var

needle.Tensor([0.91912496 1.166425   1.0383     1.1388249 ])

In [217]:
tuple((1, 2))

(1, 2)

In [218]:
def tuplify(obj):
    """Convert `obj` to tuple."""
    if isinstance(obj, tuple):
        return obj
    return tuple(listify(obj))

In [220]:
ndl.nn.Sequential(nn.Linear(10, 10)).parameters()

[]

In [216]:
y_ndl.backward()

ValueError: cannot reshape array of size 5 into shape (4,)

In [214]:
x_torch = torch.tensor(x.detach().numpy(), requires_grad=True)
w_torch = torch.tensor(weight.detach().numpy(), requires_grad=True)
b_torch = torch.tensor(bias.detach().numpy(), requires_grad=True)
bn_torch = nn.BatchNorm1d(x_torch.shape[1])
bn_torch.weight.data = w_torch
bn_torch.bias.data = b_torch
# y = (bn_torch(x_torch) ** 2).sum()
y = bn_torch(x_torch)
y

In [214]:
y.backward()

In [207]:
bn_torch.running_mean

tensor([0.3250, 0.3390, 0.2110, 0.2670])

In [208]:
bn_torch.running_var

tensor([0.9191, 1.1664, 1.0383, 1.1388])

In [215]:
x_torch.grad

tensor([[ 3.8338e-03,  4.1888e-05, -1.4280e-04, -3.5575e-05],
        [-6.9319e-03, -3.1649e-05,  9.1799e-05, -8.7278e-05],
        [ 4.6576e-03, -8.9362e-05, -4.5389e-05,  4.2690e-05],
        [-7.7194e-03,  2.9322e-05,  6.7319e-05,  7.8740e-05],
        [ 6.1755e-03,  5.0266e-05,  2.6265e-05, -6.3739e-07]])

# LayerNorm

In [48]:
x = get_tensor(5, 4)
layernorm = ndl.nn.LayerNorm1d(4)
y_ndl = layernorm(x)
y_ndl.backward()
y_ndl, x.grad

(needle.Tensor([[ 0.58616877  1.242678   -1.3833588  -0.4454885 ]
  [ 0.5073314   0.21035685  0.95279324 -1.670482  ]
  [ 0.92134947 -1.2319168  -0.7350092   1.0455765 ]
  [-1.1089483   0.7175546  -0.84801936  1.2394128 ]
  [ 0.28115013  1.5128565  -1.0041085  -0.78989875]]),
 needle.Tensor([[ 1.1920929e-07  2.3841858e-07 -2.9802322e-07 -1.1920929e-07]
  [ 1.7881393e-07  5.9604645e-08  3.5762787e-07 -5.9604645e-07]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
  [-3.5762787e-07  2.3841858e-07 -3.5762787e-07  4.7683716e-07]
  [ 2.3841858e-07  8.3446503e-07 -3.5762787e-07 -2.3841858e-07]]))

In [78]:
x_torch = torch.tensor(x.detach().numpy(), requires_grad=True)
layernorm_torch = nn.LayerNorm(x_torch.shape[1])
y = layernorm_torch(x_torch)
y.backward(torch.ones_like(y))
y, x_torch.grad

(tensor([[-0.7759, -0.5554,  0.6718,  0.7381,  1.4245, -1.3414,  1.7393, -0.5451,
          -0.7727, -0.5832]], grad_fn=<NativeLayerNormBackward0>),
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))

In [79]:
layernorm_torch.weight

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], requires_grad=True)

In [54]:
np.allclose(layernorm_torch.weight.data, layernorm.weight.cached_data)

True

In [55]:
np.allclose(layernorm_torch.bias.data, layernorm.bias.cached_data)

True

In [60]:
x = get_tensor(4, 1)
layernorm = ndl.nn.LayerNorm1d(1)
y_ndl = layernorm(x)
y_ndl.backward()
y_ndl, x.grad

(needle.Tensor([[0.]
  [0.]
  [0.]
  [0.]]),
 needle.Tensor([[0.]
  [0.]
  [0.]
  [0.]]))

In [61]:
x_torch = torch.tensor(x.detach().numpy(), requires_grad=True)
layernorm_torch = nn.LayerNorm(x_torch.shape[1])
y = layernorm_torch(x_torch)
y.backward(torch.ones_like(y))
y, x_torch.grad

(tensor([[0.],
         [0.],
         [0.],
         [0.]], grad_fn=<NativeLayerNormBackward0>),
 tensor([[0.],
         [0.],
         [0.],
         [0.]]))

In [54]:
np.allclose(layernorm_torch.weight.data, layernorm.weight.cached_data)

True

In [55]:
np.allclose(layernorm_torch.bias.data, layernorm.bias.cached_data)

True

# Broadcast/Reshape/Summation

## PowerScalar

In [17]:
x = torch.rand(1, 10, requires_grad=True)
y = x**0.5
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = x_ndl**0.5
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

In [18]:
x = torch.rand(1, 10, requires_grad=True)
y = x**2
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = x_ndl**2
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

## Broadcast

In [19]:
x = torch.rand(1, 10, requires_grad=True)
y = x.broadcast_to(5, 10)
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.broadcast_to(x_ndl, (5, 10))
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

In [20]:
x = torch.rand(1, 10, 1, requires_grad=True)
y = x.broadcast_to(5, 10, 20)
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.broadcast_to(x_ndl, (5, 10, 20))
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

In [21]:
x = torch.rand(1, 10, 1, requires_grad=True)
y = x.broadcast_to(1, 10, 20)
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.broadcast_to(x_ndl, (1, 10, 20))
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

## Reshape

In [22]:
x = torch.rand(10, requires_grad=True)
y = x.reshape(1, 10)
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.reshape(x_ndl, (1, 10))
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

In [23]:
x = torch.rand(10, requires_grad=True)
y = x.reshape(1, 1, 10)
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.reshape(x_ndl, (1, 1, 10))
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

In [24]:
x = torch.rand(10, requires_grad=True)
y = x.reshape(1, 10, 1)
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.reshape(x_ndl, (1, 10, 1))
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

## Broadcast & Reshape

In [25]:
x = torch.rand(10, requires_grad=True)
y = torch.broadcast_to(torch.reshape(x, (1, 10)), (5, 10))
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.broadcast_to(ndl.ops.reshape(x_ndl, (1, 10)), (5, 10))
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

In [26]:
x = torch.rand(10, requires_grad=True)
y = torch.broadcast_to(torch.reshape(x, (1, 10, 1)), (1, 10, 20))
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.broadcast_to(ndl.ops.reshape(x_ndl, (1, 10, 1)), (1, 10, 20))
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

## Summation

In [27]:
x = torch.rand(5, 10, requires_grad=True)
y = x.sum(0)
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.summation(x_ndl, 0)
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

In [28]:
x = torch.rand(5, 10, requires_grad=True)
y = x.sum(1)
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.summation(x_ndl, 1)
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

In [29]:
x = torch.rand(5, 10, requires_grad=True)
y = x.sum()
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.summation(x_ndl)
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

## ReLU

In [38]:
x = torch.randn(5, 10, requires_grad=True)
y = nn.ReLU()(x)
y.backward(torch.ones_like(y))

x_ndl = Tensor(x.detach().numpy())
y_ndl = ndl.ops.relu(x_ndl)
y_ndl.backward()

np.allclose(x.grad.data, x_ndl.grad.cached_data)

True

In [39]:
x.grad.data

tensor([[0., 0., 0., 1., 1., 0., 1., 0., 0., 1.],
        [1., 1., 1., 1., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 1., 1., 1.],
        [0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 1., 0., 1., 1., 1., 1.]])

In [40]:
x_ndl.grad.cached_data

array([[0., 0., 0., 1., 1., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 1., 1., 1.],
       [0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 1., 1., 0., 1., 1., 1., 1.]], dtype=float32)

# Transformations/Data

In [33]:
x_np = x.detach().numpy()
x_np

array([[0.5565526 , 0.44532466, 0.8625604 , 0.11051959, 0.7719575 ,
        0.45706648, 0.79216236, 0.9238836 , 0.5096416 , 0.8624742 ]],
      dtype=float32)

In [34]:
np.flip(x_np)

array([[0.8624742 , 0.5096416 , 0.9238836 , 0.79216236, 0.45706648,
        0.7719575 , 0.11051959, 0.8625604 , 0.44532466, 0.5565526 ]],
      dtype=float32)

In [38]:
a = torch.tensor([[[0.3994989103160902, 0.30183087592751656], [0.7752197774666111, 0.9262129301763149], [0.3253103051589964, 0.9528706723868426]], [[0.013948395933415347, 0.5334656817509289], [0.3045820306759077, 0.882859998043791], [0.25062272019388543, 0.6774118294719442]], [[0.8104240912598908, 0.4321480499248166], [0.7521345209879338, 0.8296022479277191], [0.37903377210381817, 0.09654960982745808]], [[0.25613957741471227, 0.5919351581884497], [0.47647720054983067, 0.4879345689871769], [0.4585145522366506, 0.5245926207548479]]])
b = torch.tensor([[[0.3253103051589964, 0.9528706723868426], [0.7752197774666111, 0.9262129301763149], [0.3994989103160902, 0.30183087592751656]], [[0.25062272019388543, 0.6774118294719442], [0.3045820306759077, 0.882859998043791], [0.013948395933415347, 0.5334656817509289]], [[0.37903377210381817, 0.09654960982745808], [0.7521345209879338, 0.8296022479277191], [0.8104240912598908, 0.4321480499248166]], [[0.4585145522366506, 0.5245926207548479], [0.47647720054983067, 0.4879345689871769], [0.25613957741471227, 0.5919351581884497]]])
# np.testing.assert_allclose(tform(a), b)

In [43]:
a

tensor([[[0.3995, 0.3018],
         [0.7752, 0.9262],
         [0.3253, 0.9529]],

        [[0.0139, 0.5335],
         [0.3046, 0.8829],
         [0.2506, 0.6774]],

        [[0.8104, 0.4321],
         [0.7521, 0.8296],
         [0.3790, 0.0965]],

        [[0.2561, 0.5919],
         [0.4765, 0.4879],
         [0.4585, 0.5246]]])

In [44]:
b

tensor([[[0.3253, 0.9529],
         [0.7752, 0.9262],
         [0.3995, 0.3018]],

        [[0.2506, 0.6774],
         [0.3046, 0.8829],
         [0.0139, 0.5335]],

        [[0.3790, 0.0965],
         [0.7521, 0.8296],
         [0.8104, 0.4321]],

        [[0.4585, 0.5246],
         [0.4765, 0.4879],
         [0.2561, 0.5919]]])

In [64]:
a.detach().numpy().transpose((2, 0, 1))

array([[[0.3994989 , 0.7752198 , 0.32531032],
        [0.0139484 , 0.30458203, 0.25062272],
        [0.8104241 , 0.7521345 , 0.37903377],
        [0.25613958, 0.4764772 , 0.45851454]],

       [[0.3018309 , 0.9262129 , 0.95287067],
        [0.5334657 , 0.88286   , 0.67741185],
        [0.43214804, 0.82960224, 0.09654961],
        [0.59193516, 0.48793456, 0.52459264]]], dtype=float32)

In [66]:
RandomHorizontalFlip()(torch.tensor(a.detach().numpy().transpose((2, 0, 1))))

tensor([[[0.3995, 0.7752, 0.3253],
         [0.0139, 0.3046, 0.2506],
         [0.8104, 0.7521, 0.3790],
         [0.2561, 0.4765, 0.4585]],

        [[0.3018, 0.9262, 0.9529],
         [0.5335, 0.8829, 0.6774],
         [0.4321, 0.8296, 0.0965],
         [0.5919, 0.4879, 0.5246]]])

In [67]:
a

tensor([[[0.3995, 0.3018],
         [0.7752, 0.9262],
         [0.3253, 0.9529]],

        [[0.0139, 0.5335],
         [0.3046, 0.8829],
         [0.2506, 0.6774]],

        [[0.8104, 0.4321],
         [0.7521, 0.8296],
         [0.3790, 0.0965]],

        [[0.2561, 0.5919],
         [0.4765, 0.4879],
         [0.4585, 0.5246]]])

In [73]:
np.flip(a.detach().numpy(), 1)

array([[[0.32531032, 0.95287067],
        [0.7752198 , 0.9262129 ],
        [0.3994989 , 0.3018309 ]],

       [[0.25062272, 0.67741185],
        [0.30458203, 0.88286   ],
        [0.0139484 , 0.5334657 ]],

       [[0.37903377, 0.09654961],
        [0.7521345 , 0.82960224],
        [0.8104241 , 0.43214804]],

       [[0.45851454, 0.52459264],
        [0.4764772 , 0.48793456],
        [0.25613958, 0.59193516]]], dtype=float32)

In [37]:
b

tensor([[[0.3253, 0.9529],
         [0.7752, 0.9262],
         [0.3995, 0.3018]],

        [[0.2506, 0.6774],
         [0.3046, 0.8829],
         [0.0139, 0.5335]],

        [[0.3790, 0.0965],
         [0.7521, 0.8296],
         [0.8104, 0.4321]],

        [[0.4585, 0.5246],
         [0.4765, 0.4879],
         [0.2561, 0.5919]]])

In [55]:
A = np.arange(8).reshape((2,2,2))
A

array([[[0, 1],
        [2, 3]],

       [[4, 5],
        [6, 7]]])

In [56]:
np.flip(A, -1)

array([[[1, 0],
        [3, 2]],

       [[5, 4],
        [7, 6]]])

In [133]:
h, w, c = a.shape
pad = np.pad(a.detach().numpy(), pad_width=((3, 3), (3, 3), (0, 0)))
pad.shape

(10, 9, 2)

In [134]:
h, w, c

(4, 3, 2)

In [135]:
shift_x, shift_y = np.random.randint(low=-3, high=3 + 1, size=2)
shift_x, shift_y

(0, -2)

In [136]:
top_left = 3 + shift_x
bottom_left = 3 + shift_y
top_left, bottom_left

(3, 1)

In [137]:
pad[top_left : top_left + h, bottom_left : bottom_left + w, :].shape

(4, 3, 2)

In [138]:
pad[top_left : top_left + h, bottom_left : bottom_left + w, :]

array([[[0.        , 0.        ],
        [0.        , 0.        ],
        [0.3994989 , 0.3018309 ]],

       [[0.        , 0.        ],
        [0.        , 0.        ],
        [0.0139484 , 0.5334657 ]],

       [[0.        , 0.        ],
        [0.        , 0.        ],
        [0.8104241 , 0.43214804]],

       [[0.        , 0.        ],
        [0.        , 0.        ],
        [0.25613958, 0.59193516]]], dtype=float32)

In [130]:
x = np.ones((3, 3, 3))
y = np.pad(x, ((1, 1), (1, 1), (0, 0)))
y.shape

(5, 5, 3)

In [141]:
pwd

'/Users/imad/Documents/dl-systems'

In [143]:
mnist_train_dataset = ndl.data.MNISTDataset(
    "hw2/data/train-images-idx3-ubyte.gz", "hw2/data/train-labels-idx1-ubyte.gz"
)

In [151]:
len(mnist_train_dataset[69][:25])

69


2

In [169]:
d = mnist_train_dataset[822]

822


In [170]:
d[:-25]

()

In [172]:
list(range(10, 100, 10))

[10, 20, 30, 40, 50, 60, 70, 80, 90]

9

In [175]:
np.array_split(np.arange(100), range(10, 100, 10))

[autoreload of needle.data failed: Traceback (most recent call last):
  File "/Users/imad/anaconda3/envs/dl/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/imad/anaconda3/envs/dl/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/imad/anaconda3/envs/dl/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/Users/imad/anaconda3/envs/dl/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 329, in update_class
    if update_generic(old_obj, new_obj):
  File "/Users/imad/anaconda3/envs/dl/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/Users/imad/anaconda3/envs/dl/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 289, in update_function
    se

[array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
 array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39]),
 array([40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
 array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59]),
 array([60, 61, 62, 63, 64, 65, 66, 67, 68, 69]),
 array([70, 71, 72, 73, 74, 75, 76, 77, 78, 79]),
 array([80, 81, 82, 83, 84, 85, 86, 87, 88, 89]),
 array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99])]

In [178]:
a = np.arange(100)
np.random.shuffle(a)
a

array([82,  5, 53, 69, 73, 74, 27, 31, 23,  0, 62, 98,  4,  3, 49, 32,  2,
        1, 65, 63, 61, 64, 24, 56, 58, 40,  9, 45, 66, 71, 72, 78, 88, 93,
       36, 81, 80, 57, 76, 52, 68, 28, 34, 42, 18, 16, 87, 90, 37, 39, 25,
       60,  6, 12, 46, 11, 35, 79, 43, 48, 92, 15, 30, 99, 26, 97, 21, 17,
        8, 70, 51, 59, 75, 50, 94, 89, 96, 84, 13, 86, 10, 22, 83, 20, 41,
       77, 38, 33, 44, 54, 29, 91, 55, 14,  7, 19, 67, 95, 85, 47])

# SGD

In [ ]:
learn_model_1d(
    64,
    16,
    lambda z: nn.Sequential(
        nn.Linear(64, 32), nn.ReLU(), nn.Linear(32, 16)
    ),
    ndl.optim.SGD,
    lr=0.01,
    momentum=0.0,
)
np.array(3.207009)

In [388]:
w = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]], dtype="float64")
b = np.array([[1.0, 2.0, 3.0]], dtype="float64")

In [389]:
np.random.seed(42)
# model = ndl.nn.Sequential(ndl.nn.Linear(64, 32), ndl.nn.ReLU(), ndl.nn.Linear(32, 16))
model = ndl.nn.Linear(3, 3)
model.weight.cached_data = w
model.bias.cached_data = b
X = get_tensor(1024, 3).cached_data
y = get_int_tensor(1024, low=0, high=3).cached_data.astype(np.uint8)
m = X.shape[0]
batch = 32

loss_func = ndl.nn.SoftmaxLoss()
opt = ndl.optim.Adam(model.parameters(), lr=0.001)
X, y = ndl.Tensor(X, dtype="float32"), ndl.Tensor(y)
opt.reset_grad()
out = model(X)
loss = loss_func(out, y)
loss.backward()
opt.step()
# for _ in range(1):
#     for i, (X0, y0) in enumerate(
#         zip(np.array_split(X, m // batch), np.array_split(y, m // batch))
#     ):
#         opt.reset_grad()
#         X0, y0 = ndl.Tensor(X0, dtype="float32"), ndl.Tensor(y0)
#         out = model(X0)
#         print(torch.tensor(out.detach().numpy()).norm().item(), end="\t")
#         loss = loss_func(out, y0)
#         print(loss)
#         loss.backward()
#         opt.step()

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
[[1.001 2.001 2.999]
 [4.001 5.001 5.999]
 [7.001 8.001 8.999]]
[[1. 2. 3.]]
[[1.001 2.001 2.999]]


In [390]:
model.weight.grad

needle.Tensor([[-0.89875501 -0.84187789  1.74063291]
 [-0.8917634  -0.86913744  1.76090084]
 [-0.91356435 -0.8470975   1.76066185]])

In [391]:
model.bias.grad

needle.Tensor([[-0.36216823 -0.34370205  0.70587028]])

In [392]:
model.weight.data

needle.Tensor([[1.001 2.001 2.999]
 [4.001 5.001 5.999]
 [7.001 8.001 8.999]])

In [393]:
model.bias.data

needle.Tensor([[1.001 2.001 2.999]])

In [394]:
w = torch.tensor(
    [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]], dtype=torch.float64
)
b = torch.tensor([[1.0, 2.0, 3.0]], dtype=torch.float64)

In [395]:
np.random.seed(42)
# model = nn.Sequential(nn.Linear(64, 32), nn.ReLU(), nn.Linear(32, 16))
model = nn.Linear(3, 3)
model.weight = nn.Parameter(w.T)
model.bias = nn.Parameter(b)
loss_func = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr=0.001)
X, y = torch.tensor(X.detach().numpy(), dtype=torch.float64), torch.tensor(
    y.detach().numpy()
)
# out0 = torch.tensor(X) @ w + b
opt.zero_grad()
out1 = model(X)
loss = loss_func(out1, y)
loss.backward()
opt.step()
# for _ in range(1):
#     for i, (X0, y0) in enumerate(
#         zip(np.array_split(X, m // batch), np.array_split(y, m // batch))
#     ):
#         opt.zero_grad()
#         X0, y0 = torch.tensor(X0, dtype=torch.float64), torch.tensor(y0)
#         out = model(X0)
#         print(out.norm().item(), end="\t")
#         loss = loss_func(out, y0)
#         print(loss.item())
#         loss.backward()
#         opt.step()

In [396]:
model.weight.grad.T

tensor([[-0.8988, -0.8419,  1.7406],
        [-0.8918, -0.8691,  1.7609],
        [-0.9136, -0.8471,  1.7607]], dtype=torch.float64)

In [397]:
model.bias.grad

tensor([[-0.3622, -0.3437,  0.7059]], dtype=torch.float64)

In [398]:
model.weight.data.T

tensor([[1.0009, 2.0008, 2.9983],
        [4.0009, 5.0009, 5.9982],
        [7.0009, 8.0008, 8.9982]], dtype=torch.float64)

In [399]:
model.bias.data

tensor([[1.0004, 2.0003, 2.9993]], dtype=torch.float64)